In [ ]:
pip install transformers

## Import librarys

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

### Load Image

In [ ]:
img_url = '/content/2.jpg'
raw_image = Image.open(img_url).convert("RGB")
image = cv2.imread(img_url)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load all Models

## Segment Anything Model 

In [ ]:

import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'


import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

#### Create image from mask

In [ ]:
from PIL import Image

def mask_image(mask, raw_image):
    weigth, heigth = raw_image.size
    new_image = Image.new('RGBA', (weigth, heigth), (0, 0, 0, 0))

    original_pixles = raw_image.load()
    pixels = new_image.load()

    for i in range (heigth):
        for j in range (weigth):
            if mask[i,j]:
                pixels[j, i] = original_pixles[j,i]
            else:
                pass
    return new_image

def bbox_image(bbox, image):
    x,y,w,h =  bbox[0],bbox[1],bbox[2],bbox[3]
    return image[y:y+h, x:x+w]

In [ ]:
def all_areas_from_image(image, raw_image):
    masks = mask_generator.generate(image)
    images_box= []
    images_mask= []
    for mask in masks:
        images_box.append(bbox_image(mask['bbox'],image))
        images_mask.append(mask_image(mask['segmentation'], raw_image))
    return {'box':images_box, 'mask':images_mask}

## BLIP

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

def blip (_image):
    inputs = processor(_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    print(processor.decode(out[0], skip_special_tokens=True))
    return processor.decode(out[0], skip_special_tokens=True)

def all_captions(image, raw_image):
    areas = all_areas_from_image(image, raw_image)['mask']
    origin = str(blip(image))
    captions = [origin]
    for im in areas:
        captions.append(origin + str(blip(im)))
    return captions    

## CLIP

In [ ]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
def select_caption(captions, image):
    inputs = processor(text=captions, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
    print(probs)
    return select_from_probs(probs, captions)

def select_from_probs(probs, captions):
    max_prob = 0
    index = 0
    for i,prob in zip(range(len(probs)),probs):
        if prob > max_prob:
            max_prob = prob
            index = i
    return captions[index]

# Run Model

In [ ]:
captions = all_areas_from_image(image, raw_image)
for caption in captions:
    print(caption)
    
result = select_caption(captions, image)
print(result)